# Document Clustering

In [1]:
import os
import numpy as np
import pandas as pd
import sys
sys.path.append(os.path.abspath(".."))

import plotly.plotly as py
from plotly.graph_objs import *
from url2vec.util.plotter import *
from url2vec.util.seqmanager import *

from sklearn import metrics
from hdbscan import HDBSCAN
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from __future__ import print_function
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

---

In [2]:
# cs.illinois.edu    cs.stanford.edu    eecs.mit.edu
site = "cs.stanford.edu"

## TF-IDF matrix

Here is defined **term frequency - inverse document frequency** (tf-idf) vectorizer parameters and then convert the documents (web pages) list into a tf-idf matrix.

To get a Tf-idf matrix, first count word occurrences by document. This is transformed into a **document-term matrix** (dtm).![Alt text](http://www.codeproject.com/KB/WPF/NNMFSearchResultClusterin/table.jpg "Very nice")

This is also just called a term frequency matrix.
Then apply the term frequency-inverse document frequency weighting: words that occur frequently within a document but not frequently within the corpus receive a higher weighting as these words are assumed to contain more meaning in relation to the document.

A couple things to note about the parameters defined below:

**max_df**: this is the maximum frequency within the documents a given feature can have to be used in the tfi-idf matrix. If the term is in greater than 80% of the documents it probably cares little meanining

**min_idf**: this could be an integer (e.g. 5) and the term would have to be in at least 5 of the documents to be considered. Here I pass 0.1; the term must be in at least 10% of the document.

**ngram_range**: this just means I'll look at unigrams, bigrams and trigrams.

In [3]:
tfidf_vectorizer = TfidfVectorizer(
    max_df = 0.8,
    max_features = 200000,
    min_df = 0.1,
    stop_words = 'english',
    use_idf = True,
    tokenizer = tokenize_and_stem,
    ngram_range = (1,3)
)

The crawling proccess has been done in two different ways:

- **No costraint**: the crawler follows a random outlink from all of the outlinks in a given page
- **List costraint**: the crawler follows a random outlink but only from the outlinks in "lists"

## No-costraint documents

In [4]:
nocostraint_path = os.getcwd() + "/../dataset/" + site + "/no_constraint/words1000_depth10/"
vertex_nc_path   = nocostraint_path + "vertex.txt"
map_nc_path      = nocostraint_path + "urlsMap.txt"

codecontent_map_nc = get_content_map(vertex_nc_path)
urlmap_nc          = get_urlmap(map_nc_path)

documents_nc = [codecontent_map_nc[key] for key in codecontent_map_nc]
codes_nc     = [key for key in codecontent_map_nc]
urls_nc      = [urlmap_nc[key] for key in codecontent_map_nc]

#### Cosine-Similarity
Cosine similarity is measured against the tf-idf matrix and can be used to generate a measure of similarity between each document and the other documents in the corpus.

Subtracting it from 1 provides cosine distance which I will use for plotting on a euclidean (2-dimensional) plane.

In [5]:
tfidf_matrix_nc = tfidf_vectorizer.fit_transform(documents_nc)
dist_nc = 1 - cosine_similarity(tfidf_matrix_nc)

print(pd.DataFrame(
        {"documents":tfidf_matrix_nc.shape[0], "terms":tfidf_matrix_nc.shape[1]}, 
        index=[""]).T)
print("\n\n")

               
documents  1458
terms       843





## Clustering on No-costraint documents
#### K-Means

K-means initializes with a pre-determined number of clusters. Each observation is assigned to a cluster (cluster assignment) so as to minimize the within cluster sum of squares. Next, the mean of the clustered observations is calculated and used as the new cluster centroid. Then, observations are reassigned to clusters and centroids recalculated in an iterative process until the algorithm reaches convergence.

In [6]:
kmeans = KMeans(n_clusters=15)
kmeans_labels_nc = kmeans.fit_predict(tfidf_matrix_nc)

docs_nc = { 
    'code': codes_nc,
    'document': documents_nc
}
frame_nc = pd.DataFrame(docs_nc, index=[kmeans_labels_nc] , columns=['document', 'code'])

frame_nc[:5]

,document,code
2,recurrentjs sentence memorization demo deep re...,344
6,reinforcejs gridworld with dynamic programming...,345
10,tsnejs demo tsnejs demo intro t sne is a visua...,346
6,nips accepted papers nips papers in nicer form...,347
10,z cmpt home publications cv contact warning th...,340


#### Topic modeling
Some fancy indexing and sorting on each cluster to identify which are the top n words that are nearest to the cluster centroid. This gives an idea of the main topic of each the cluster.

In [7]:
# map -> {code: token_list}
tokens_nc_map = to_tokens_map(codecontent_map_nc)
# map -> {code: stem_list}
stems_nc_map = to_stems_map(codecontent_map_nc)

# total vocabulary, list of tokens
totalvocab_nc_stemmed = [stem for key in codecontent_map_nc for stem in stems_nc_map[key]]
# total vocabulary, list of stems
totalvocab_nc_tokenized = [stem for key in codecontent_map_nc for stem in tokens_nc_map[key]]

vocab_nc_frame = pd.DataFrame({'words': totalvocab_nc_tokenized}, index = totalvocab_nc_stemmed)
terms_nc = tfidf_vectorizer.get_feature_names()

# sort cluster centers by proximity to centroid
order_centroids_nc = kmeans.cluster_centers_.argsort()[:,::-1]

num_clusters_nc = len(set(kmeans_labels_nc))
words_matrix_nc = [None] * num_clusters_nc
top_n = 7

for i in range(num_clusters_nc):
    cluster_chart = [vocab_nc_frame.ix[terms_nc[ind].split(' ')].values.tolist()[0][0] 
                     for ind in order_centroids_nc[i,:top_n]]
    words_matrix_nc[i] = cluster_chart
    
pd.DataFrame(
    words_matrix_nc, 
    index = ["Cluster " + str(i) + " - Top Words" for i in range(num_clusters_nc)],
    columns = list(range(1, top_n+1))
)

,1,2,3,4,5,6,7
Cluster 0 - Top Words,student,cs,programming,science,computations,degrees,ms
Cluster 1 - Top Words,pm,pm,cs,student,ms,access,masters
Cluster 2 - Top Words,t,day,time,just,very,good,hour
Cluster 3 - Top Words,student,cs,ms,access,masters,degrees,computations
Cluster 4 - Top Words,paper,intelligence,publications,international,learn,project,code
Cluster 5 - Top Words,t,w,m,student,cs,ms,access
Cluster 6 - Top Words,learn,models,networks,control,machine,code,algorithms
Cluster 7 - Top Words,webauth,page,machine,login,webauth,csid,cs
Cluster 8 - Top Words,cs,list,b,project,course,programming,please
Cluster 9 - Top Words,robotics,control,human,designed,project,learn,m


### K-Means Plot
Applying t-SNE for dimensionality reduction. We need two dimensional vectors for visualization purposes.

In [8]:
tsne = TSNE(n_components=2, random_state=1)
twodim_docs_nc = tsne.fit_transform(dist_nc)

clusters_colors_nc = [ get_color(i) for i in kmeans_labels_nc]

kmeans_data = scatter_plot(twodim_docs_nc, word_labels=urls_nc, colors=clusters_colors_nc)
py.iplot(kmeans_data, filename="K-Means t-SNE nocostraint - Doc Clustering")

<div>
    <a href="https://plot.ly/~chrispolo/70" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/nc_docs_wordvectors_scatter_plot_KMEANS.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:70"  src="https://plot.ly/embed.js" async></script>
</div>

---

## List-costraint documents

In [9]:
listcostraint_path = os.getcwd() + "/../dataset/" + site + "/list_constraint/words1000_depth10/"
vertex_lc_path     = listcostraint_path + "vertex.txt"
map_lc_path        = listcostraint_path + "urlsMap.txt"

codecontent_map_lc = get_content_map(vertex_lc_path)
urlmap_lc = get_urlmap(map_lc_path)

# document list
documents_lc = [codecontent_map_lc[key] for key in codecontent_map_lc]
codes_lc     = [key for key in codecontent_map_lc]
urls_lc      = [urlmap_lc[key] for key in codecontent_map_lc]

#### Cosine-Similarity
Cosine similarity is measured against the tf-idf matrix and can be used to generate a measure of similarity between each document and the other documents in the corpus.

Subtracting it from 1 provides cosine distance which I will use for plotting on a euclidean (2-dimensional) plane.

In [10]:
tfidf_matrix_lc = tfidf_vectorizer.fit_transform(documents_lc)

dist_lc = 1 - cosine_similarity(tfidf_matrix_lc)

print(pd.DataFrame({"documents":tfidf_matrix_lc.shape[0], "terms":tfidf_matrix_lc.shape[1]}, index=[""]).T)
print("\n\n")

              
documents  152
terms      330





## Clustering on List-costraint documents
#### K-Means
K-means initializes with a pre-determined number of clusters. Each observation is assigned to a cluster (cluster assignment) so as to minimize the within cluster sum of squares. Next, the mean of the clustered observations is calculated and used as the new cluster centroid. Then, observations are reassigned to clusters and centroids recalculated in an iterative process until the algorithm reaches convergence.

In [11]:
kmeans = KMeans(n_clusters=15)
kmeans_labels_lc = kmeans.fit_predict(tfidf_matrix_lc)
docs_lc = {
    'code': codes_lc,
    'document': documents_lc
}

frame_lc = pd.DataFrame(docs_lc, index = [kmeans_labels_lc] , columns = ['document', 'code'])
frame_lc[:5]

,document,code
4,skip to skip to content skip to navigation csi...,216
1,skip to skip to content skip to navigation csi...,214
4,skip to skip to content skip to navigation csi...,215
9,skip to skip to content skip to navigation csi...,212
4,skip to skip to content skip to navigation csi...,213


#### Topic modeling
Some fancy indexing and sorting on each cluster to identify which are the top n words that are nearest to the cluster centroid. This gives an idea of the main topic of each the cluster.

In [12]:
# map -> {code: token_list}
tokens_lc_map = to_tokens_map(codecontent_map_lc)
# map -> {code: stem_list}
stems_lc_map = to_stems_map(codecontent_map_lc)

# total vocabulary, list of tokens
totalvocab_lc_stemmed = [stem for key in codecontent_map_lc for stem in stems_lc_map[key]]
# total vocabulary, list of stems
totalvocab_lc_tokenized = [stem for key in codecontent_map_lc for stem in tokens_lc_map[key]]

vocab_lc_frame = pd.DataFrame({'words': totalvocab_lc_tokenized}, index = totalvocab_lc_stemmed)
terms_lc = tfidf_vectorizer.get_feature_names()

# sort cluster centers by proximity to centroid
order_centroids_lc = kmeans.cluster_centers_.argsort()[:, ::-1]

num_clusters_lc = len(set(kmeans_labels_lc))
words_matrix_lc = [None] * num_clusters_lc
top_n = 7

for i in range(num_clusters_lc):
    cluster_chart = [vocab_lc_frame.ix[terms_lc[ind].split(' ')].values.tolist()[0][0] for ind in order_centroids_lc[i,:top_n]]
    words_matrix_lc[i] = cluster_chart
    
pd.DataFrame(
    words_matrix_lc, 
    index = ["Cluster " + str(i) + " - Top Words" for i in range(num_clusters_lc)],
    columns = list(range(1, top_n+1))
)

,1,2,3,4,5,6,7
Cluster 0 - Top Words,control,teaching,combination,links,fields,dynamical,updated
Cluster 1 - Top Words,application,deadlines,online,please,submitting,admissions,ph
Cluster 2 - Top Words,list,b,project,p,practical,training,programming
Cluster 3 - Top Words,professor,professor,faculty,secure,humanities,focusing,computer
Cluster 4 - Top Words,admissions,skip,computer,search,campus,gates,building
Cluster 5 - Top Words,motions,learn,robotics,dynamical,demonstrated,modeling,based
Cluster 6 - Top Words,robotics,design,humanities,control,receive,template,professor
Cluster 7 - Top Words,page,served,web,directed,help,machine,use
Cluster 8 - Top Words,programming,student,courses,degree,ms,school,requires
Cluster 9 - Top Words,cas,courses,ca,resources,teaching,time,quarterly


### K-Means Plot
Applying t-SNE for dimensionality reduction. We need two dimensional vectors for visualization purposes.

In [13]:
tsne = TSNE(n_components=2, random_state=1)
twodim_docs_lc = tsne.fit_transform(dist_lc)

clusters_colors_lc = [ get_color(i) for i in kmeans_labels_lc]

k_tsne_data_lc = scatter_plot(twodim_docs_lc, word_labels=urls_lc, colors=clusters_colors_lc)
py.iplot(k_tsne_data_lc, filename="K-Means listcostraint - Doc Clustering")

<div>
    <a href="https://plot.ly/~chrispolo/74" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/lc_docs_wordvectors_scatter_plot_KMEANS.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:74"  src="https://plot.ly/embed.js" async></script>
</div>

---

## Evaluation
Evaluating the performance of a clustering algorithm is not as trivial as counting the number of errors or the precision and recall of a supervised classification algorithm. In particular any evaluation metric should not take the absolute values of the cluster labels into account but rather if this clustering define separations of the data similar to some ground truth set of classes or satisfying some assumption such that members belong to the same class are more similar that members of different classes according to some similarity metric.

See the [scikit-learn documentaion](http://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation "ti") for futher information

### Ground Truth

In [14]:
gt = GroundTruth(os.getcwd() + "/../dataset/" + site + "/ground_truth/urlToMembership.txt")
ground_truth_lc = [int(gt.get_groundtruth(urlmap_lc[key])) for key in codecontent_map_lc]

gt = GroundTruth(os.getcwd() + "/../dataset/" + site + "/ground_truth/urlToMembership.txt")
ground_truth_nc = [int(gt.get_groundtruth(urlmap_nc[key])) for key in codecontent_map_nc]

print("Clusters found manually for no-costraint documents:", len(set(ground_truth_nc)))
print([label for label in set(ground_truth_nc)])
print()
print("Clusters found manually for list-costraint documents:", len(set(ground_truth_lc)))
print([label for label in set(ground_truth_lc)])
print("\n\n")

Clusters found manually for no-costraint documents: 15
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -1]

Clusters found manually for list-costraint documents: 14
[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, -1]





## DBSCAN and HDBSCAN
Applying other clustering algorithm for evaluation purposes.

**DBSCAN** - Density-Based Spatial Clustering of Applications with Noise. Finds core samples of high density and expands clusters from them. Good for data which contains clusters of similar density.

**params**:

- **eps** : The maximum distance between two samples for them to be considered as in the same neighborhood.
- **min_samples** : The number of samples (or total weight) in a neighborhood for a point to be considered as a core point. This includes the point itself.


**HDBSCAN** - Hierarchical Density-Based Spatial Clustering of Applications with Noise. Performs DBSCAN over varying epsilon values and integrates the result to find a clustering that gives the best stability over epsilon. This allows HDBSCAN to find clusters of varying densities (unlike DBSCAN), and be more robust to parameter selection.

**params**:

- **min_cluster_size** : minimum nodes to form a cluster

###  No-costraint

In [15]:
dbscan = DBSCAN(eps=0.3, min_samples=5)
dbscan_labels_nc = dbscan.fit_predict(tfidf_matrix_nc)

print("Clusters found with DBSCAN:", len(set(dbscan_labels_nc)))
print ([label for label in set(dbscan_labels_nc)])
print("\n")

hdbscan = HDBSCAN(min_cluster_size=15)
hdbscan_labels_nc = hdbscan.fit_predict(tfidf_matrix_nc)

print("Clusters found with HDBSCAN:", len(set(hdbscan_labels_nc)))
print([label for label in set(hdbscan_labels_nc)])
print("\n\n")

Clusters found with DBSCAN: 11
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, -1]


Clusters found with HDBSCAN: 15
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -1]





### List-costraint

In [16]:
dbscan = DBSCAN(eps=0.5, min_samples=3)
dbscan_labels_lc = dbscan.fit_predict(tfidf_matrix_lc)

print("Clusters found with DBSCAN:", len(set(dbscan_labels_lc)))
print ([label for label in set(dbscan_labels_lc)])
print("\n")

hdbscan = HDBSCAN(min_cluster_size=5)
hdbscan_labels_lc = hdbscan.fit_predict(tfidf_matrix_lc)

print("Clusters found with HDBSCAN:", len(set(hdbscan_labels_lc)))
print([label for label in set(hdbscan_labels_lc)])
print("\n\n")

Clusters found with DBSCAN: 5
[0, 1, 2, 3, -1]


Clusters found with HDBSCAN: 4
[0, 1, 2, -1]





### Metrics:

- **Homogeneity**: each cluster contains only members of a single class


- **Completeness**: all members of a given class are assigned to the same cluster


- **Adjusted Rand index**: Given the knowledge of the *ground truth* class assignments and our clustering algorithm assignments of the same samples, the adjusted Rand index is a function that measures the similarity of the two assignments, ignoring permutations and with chance normalization


- **V-measure**: The V-measure is actually equivalent to the mutual information (NMI) discussed above normalized by the sum of the label entropies


- **Mutual Information based scores**: Given the knowledge of the ground truth class assignments and our clustering algorithm assignments of the same samples, the Mutual Information is a function that measures the agreement of the two assignments, ignoring permutations. Two different normalized versions of this measure are available, Normalized Mutual Information(NMI) and Adjusted Mutual Information(AMI). NMI is often used in the literature while AMI was proposed more recently and is normalized against chance


- **Silhouette**: If the ground truth labels are not known, evaluation must be performed using the model itself. The Silhouette Coefficient is an example of such an evaluation, where a higher Silhouette Coefficient score relates to a model with better defined clusters. The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters. The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.

In [17]:
metrics_df = pd.DataFrame([
        [
            # dbscan nocostraint
            metrics.homogeneity_score(ground_truth_nc, dbscan_labels_nc),
            metrics.completeness_score(ground_truth_nc, dbscan_labels_nc),
            metrics.v_measure_score(ground_truth_nc, dbscan_labels_nc),
            metrics.adjusted_rand_score(ground_truth_nc, dbscan_labels_nc),
            metrics.adjusted_mutual_info_score(ground_truth_nc, dbscan_labels_nc),
            metrics.silhouette_score(tfidf_matrix_nc, dbscan_labels_nc, metric='euclidean')
        ],
        [
            # hdbscan nocostraint
            metrics.homogeneity_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.completeness_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.v_measure_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.adjusted_rand_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.adjusted_mutual_info_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.silhouette_score(tfidf_matrix_nc, hdbscan_labels_nc, metric='euclidean')
        ],
        [
            # kmeans nocostraint
            metrics.homogeneity_score(ground_truth_nc, kmeans_labels_nc),
            metrics.completeness_score(ground_truth_nc, kmeans_labels_nc),
            metrics.v_measure_score(ground_truth_nc, kmeans_labels_nc),
            metrics.adjusted_rand_score(ground_truth_nc, kmeans_labels_nc),
            metrics.adjusted_mutual_info_score(ground_truth_nc, kmeans_labels_nc),
            metrics.silhouette_score(tfidf_matrix_nc, kmeans_labels_nc, metric='euclidean')
        ],
        [
            # dbscan listcostraint
            metrics.homogeneity_score(ground_truth_lc, dbscan_labels_lc),
            metrics.completeness_score(ground_truth_lc, dbscan_labels_lc),
            metrics.v_measure_score(ground_truth_lc, dbscan_labels_lc),
            metrics.adjusted_rand_score(ground_truth_lc, dbscan_labels_lc),
            metrics.adjusted_mutual_info_score(ground_truth_lc, dbscan_labels_lc),
            metrics.silhouette_score(tfidf_matrix_lc, dbscan_labels_lc, metric='euclidean')
        ],
        [
            # hdbscan listcostraint
            metrics.homogeneity_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.completeness_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.v_measure_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.adjusted_rand_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.adjusted_mutual_info_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.silhouette_score(tfidf_matrix_lc, hdbscan_labels_lc, metric='euclidean')
        ],
        [
            # kmeans listcostraint
            metrics.homogeneity_score(ground_truth_lc, kmeans_labels_lc),
            metrics.completeness_score(ground_truth_lc, kmeans_labels_lc),
            metrics.v_measure_score(ground_truth_lc, kmeans_labels_lc),
            metrics.adjusted_rand_score(ground_truth_lc, kmeans_labels_lc),
            metrics.adjusted_mutual_info_score(ground_truth_lc, kmeans_labels_lc),
            metrics.silhouette_score(tfidf_matrix_lc, kmeans_labels_lc, metric='euclidean')
        ]],
        index=[
            "NoCostraint - DBSCAN", 
            "NoCostraint - HDBSCAN", 
            "NoCostraint - K-MEANS", 
            "ListCostraint - DBSCAN", 
            "ListCostraint - HDBSCAN", 
            "ListCostraint - K-MEANS"
        ],
        columns=[
            "Homogeneity", 
            "Completeness", 
            "V-Measure score", 
            "Adjusted Rand index", 
            "Mutual Information",
            "Silhouette"
        ])

metrics_df

,Homogeneity,Completeness,V-Measure score,Adjusted Rand index,Mutual Information,Silhouette
NoCostraint - DBSCAN,0.258393,0.343509,0.294933,0.020787,0.240300,0.098145
NoCostraint - HDBSCAN,0.301496,0.278424,0.289501,0.008851,0.252128,0.091613
NoCostraint - K-MEANS,0.601448,0.465157,0.524595,0.263939,0.448893,0.270393
ListCostraint - DBSCAN,0.087603,0.392448,0.143234,0.010232,0.032697,-0.118799
ListCostraint - HDBSCAN,0.182668,0.380153,0.246763,0.068729,0.132592,0.084066
ListCostraint - K-MEANS,0.543627,0.572597,0.557736,0.216668,0.415685,0.173217
